## Contextualized model

Let's check where the context really helps

In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [16]:
%load_ext autoreload
%autoreload 2

from hatedetection import load_datasets
import glob
import json

train_dataset, dev_dataset, test_dataset = load_datasets(add_body=True)

no_context_evals = []
context_evals = []

for path in glob.glob("../evaluations/non-context*"):
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        no_context_evals.append(obj)

for path in glob.glob("../evaluations/context*"):
    with open(path) as f:
        obj = json.load(f)
        obj["file"] = path
        context_evals.append(obj)

print(f"We have {len(context_evals)} context evaluations")
print(f"We have {len(no_context_evals)} context evaluations")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
We have 10 context evaluations
We have 10 context evaluations


In [17]:
import pandas as pd

df_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in context_evals
])

df_context_evals

,file,eval_loss,eval_accuracy,eval_f1,eval_macro_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,init_mem_cpu_alloc_delta,init_mem_gpu_alloc_delta,init_mem_cpu_peaked_delta,init_mem_gpu_peaked_delta,test_mem_cpu_alloc_delta,test_mem_gpu_alloc_delta,test_mem_cpu_peaked_delta,test_mem_gpu_peaked_delta
0,../evaluations/context-6.json,0.81358,0.90611,0.67079,0.80802,0.75452,0.60378,85.52730,132.62400,51854,0,18258,0,405165,0,887161,189087744
1,../evaluations/context-3.json,0.84004,0.90391,0.66564,0.80477,0.74163,0.60378,172.47950,65.76400,51854,0,18258,0,409133,0,887288,189087744
2,../evaluations/context-4.json,0.80791,0.91025,0.68267,0.81520,0.77605,0.60935,85.04410,133.37800,51782,0,18258,0,405647,0,887233,189087744
3,../evaluations/context-7.json,0.79550,0.90770,0.68128,0.81366,0.75202,0.62270,89.38540,126.90000,51782,0,18258,0,405138,0,887161,189087744
4,../evaluations/context-1.json,0.80262,0.90505,0.66563,0.80515,0.75281,0.59655,219.52600,51.67000,51854,0,18258,0,405281,0,887225,189087744
5,../evaluations/context-2.json,0.81286,0.90576,0.66770,0.80640,0.75634,0.59766,217.89450,52.05700,51854,0,18258,0,405778,0,887161,189087744
6,../evaluations/context-9.json,0.84524,0.90241,0.65759,0.80034,0.74025,0.59154,84.45170,134.31400,51854,0,18258,0,405583,0,887161,189087744
7,../evaluations/context-10.json,0.80683,0.90188,0.66446,0.80350,0.72500,0.61324,84.97260,133.49000,51854,0,18258,0,405599,0,887224,189087744
8,../evaluations/context-8.json,0.77337,0.90408,0.66420,0.80412,0.74567,0.59878,90.74530,124.99800,51854,0,18258,0,405282,0,887161,189087744
9,../evaluations/context-5.json,0.56969,0.90602,0.67341,0.80926,0.74915,0.61157,84.75670,133.83000,51854,0,18258,0,405147,0,887224,189087744


In [18]:
metric_columns = ["eval_precision", "eval_recall", "eval_f1", "eval_macro_f1"]

pd.DataFrame({"mean": df_context_evals[metric_columns].mean(), "std": df_context_evals[metric_columns].std()})

,mean,std
eval_precision,0.74934,0.01315
eval_recall,0.60490,0.00938
eval_f1,0.66934,0.00787
eval_macro_f1,0.80704,0.00461


In [19]:
import pandas as pd

df_no_context_evals = pd.DataFrame([
    {**{"file": evaluation["file"]}, **evaluation["metrics"]} for evaluation in no_context_evals
])

pd.DataFrame({"mean": df_no_context_evals[metric_columns].mean(), "std": df_no_context_evals[metric_columns].std()})

,mean,std
eval_precision,0.67690,0.01804
eval_recall,0.59738,0.01516
eval_f1,0.63431,0.00511
eval_macro_f1,0.78509,0.00302


Hay que chequear qué pasa con los FP de no usar contexto